# M2.2 Tensorboard on Goolge Colab



In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-10-27 07:07:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.172.170.160, 52.44.92.122, 54.209.64.71, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.172.170.160|:443... connected.
HTTP request sent, awaiting response... 300 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  3.44MB/s    in 1.5s    

2018-10-27 07:07:07 (3.44 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


## CNN Prediction
- Read Training FIle
- Build Model
- Train Model
- Predict Model

In [0]:
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
DATA_PATH = "/content/gdrive/My Drive/Lecture/StudyPie/Data/"

In [5]:
!ls "/content/gdrive/My Drive/Lecture/StudyPie/Data/"

crypto_data.zip		     M2_1_X.pickle  __MACOSX   PetImages.zip
kagglecatsanddogs_3367a.zip  M2_1_y.pickle  PetImages


In [0]:
# Read training file
import pickle

pickle_in = open(DATA_PATH+"M2_1_X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open(DATA_PATH+"M2_1_y.pickle","rb")
y = pickle.load(pickle_in)

In [0]:
X = X/255.0

TRAIN_RATIO = 0.8

X_train = X[:int(len(X)*TRAIN_RATIO)]
y_train = y[:int(len(y)*TRAIN_RATIO)]

X_test = X[int(len(X)*TRAIN_RATIO):]
y_test = y[int(len(y)*TRAIN_RATIO):]

## Run TensorBoard

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

## Run ngrok

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

## Get URL

In [12]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://39eb00f7.ngrok.io


## Run a Keras model with TenorBoard

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.3,
          callbacks=[tbCallBack])

Train on 1103 samples, validate on 474 samples
Epoch 1/5
1103/1103 [==============================] - 94s 85ms/step - loss: 1.2309 - acc: 0.5113 - val_loss: 0.6916 - val_acc: 0.5359
Epoch 2/5
1103/1103 [==============================] - 89s 81ms/step - loss: 0.6824 - acc: 0.5694 - val_loss: 0.7865 - val_acc: 0.4684
Epoch 3/5
1103/1103 [==============================] - 88s 79ms/step - loss: 0.6470 - acc: 0.6437 - val_loss: 0.6634 - val_acc: 0.6118
Epoch 4/5
1103/1103 [==============================] - 87s 79ms/step - loss: 0.5694 - acc: 0.7035 - val_loss: 0.7094 - val_acc: 0.6139
Epoch 5/5
1103/1103 [==============================] - 91s 83ms/step - loss: 0.4959 - acc: 0.7561 - val_loss: 0.6908 - val_acc: 0.6498


## Optionally check tensorboard and ngrok PIDs and stop them

In [13]:
#!ps aux | grep tensorboard

root         986  0.9  3.3 1729856 445480 ?      Sl   07:07   0:04 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./log --host 0.0.0.0 --port 6006
root        1060  0.0  0.0  39196  6528 ?        S    07:15   0:00 /bin/bash -c ps aux | grep tensorboard
root        1062  0.0  0.0  39196  2696 ?        D    07:15   0:00 /bin/bash -c ps aux | grep tensorboard


In [0]:
#!kill 986

## Optionally clear the tensorboard records directory

In [0]:
#!rm -rf log